In [1]:
import glob
import os
import numpy as np
import nimblephysics as nimble
from scipy.stats import entropy

def load_joint_positions(b3d_path):
    subject = nimble.biomechanics.SubjectOnDisk(b3d_path)
    num_trials = subject.getNumTrials()
    if num_trials == 0:
        raise ValueError(f"No trials found in {b3d_path}")
    all_trials = []
    for trial_idx in range(num_trials):
        trial_length = subject.getTrialLength(trial_idx)
        frames = subject.readFrames(
            trial=trial_idx,
            startFrame=0,
            numFramesToRead=trial_length,
            includeSensorData=False,
            includeProcessingPasses=True
        )
        skeleton = subject.readSkel(processingPass=0, ignoreGeometry=True)
        if skeleton is None:
            print(f"Warning: No skeleton found for trial {trial_idx} in {b3d_path}, skipping.")
            continue
        joint_positions = []
        for frame in frames:
            dofs = frame.processingPasses[0].pos  # shape: (num_dofs,)
            skeleton.setPositions(dofs)
            positions = np.array([skeleton.getBodyNode(i).getWorldTransform().translation()
                                  for i in range(skeleton.getNumBodyNodes())])
            joint_positions.append(positions)
        if joint_positions:
            all_trials.append(np.stack(joint_positions))  # (frames, joints, 3)
    return all_trials

def compute_penetration(joint_positions):
    # joint_positions: (frames, joints, 3)
    ymin = np.min(joint_positions[..., 1], axis=1)  # y-coord of lowest joint per frame
    ground = np.median(ymin)
    penetration = np.mean(np.maximum(0, -(ymin - ground)))
    return penetration

def compute_floating(joint_positions):
    ymin = np.min(joint_positions[..., 1], axis=1)
    ground = np.median(ymin)
    floating = np.mean(np.maximum(0, ymin - ground))
    return floating

def compute_sliding(joint_positions):
    # Find lowest joint per frame
    lowest_idx = np.argmin(joint_positions[..., 1], axis=1)
    plow = np.array([joint_positions[t, idx] for t, idx in enumerate(lowest_idx)])
    sliding = np.mean(np.linalg.norm(plow[1:] - plow[:-1], axis=1))
    return sliding

def compute_entropy(joint_positions):
    # Flatten all joint positions and compute entropy on y-coords as a simple example
    y_coords = joint_positions[..., 1].flatten()
    hist, _ = np.histogram(y_coords, bins=50, density=True)
    return entropy(hist + 1e-8)  # add epsilon to avoid log(0)

def compute_fid(features_gen, features_real):
    mu_gen = np.mean(features_gen, axis=0)
    mu_real = np.mean(features_real, axis=0)
    std_gen = np.std(features_gen, axis=0)
    std_real = np.std(features_real, axis=0)
    w2 = (mu_real - mu_gen) ** 2 + (std_real - std_gen) ** 2
    return np.mean(w2)

def compute_r_precision(features_gen, features_text):
    # Placeholder: You need a retrieval setup and text features
    # Return dummy value for now
    return 0.0

def compute_metrics(b3d_dir, reference_path, match_mode="looped", group_size=3):
    b3d_files = glob.glob(os.path.join(b3d_dir, "**", "*.b3d"), recursive=True)
    penetrations, floatings, slidings, entropies, fidelities = [], [], [], [], []

    # Load all reference trials
    reference_trials = load_joint_positions(reference_path)  # List of arrays, length N

    for b3d_path in b3d_files:
        try:
            all_trials = load_joint_positions(b3d_path)
        except Exception as e:
            print(f"Skipping {b3d_path}: {e}")
            continue
        for trial_idx, joint_positions in enumerate(all_trials):
            if joint_positions.ndim != 3 or joint_positions.shape[0] == 0:
                print(f"Skipping {b3d_path} trial {trial_idx}: invalid joint_positions shape {joint_positions.shape}")
                continue

            if match_mode == "looped":
                ref_idx = trial_idx % len(reference_trials)
            elif match_mode == "grouped":
                ref_idx = trial_idx // group_size
                if ref_idx >= len(reference_trials):
                    print(f"Skipping trial {trial_idx}: ref_idx {ref_idx} out of bounds")
                    continue
            else:
                raise ValueError(f"Unknown match_mode: {match_mode}")

            reference_positions = reference_trials[ref_idx]
            reference_features = reference_positions.reshape(reference_positions.shape[0], -1)

            penetrations.append(compute_penetration(joint_positions))
            floatings.append(compute_floating(joint_positions))
            slidings.append(compute_sliding(joint_positions))
            entropies.append(compute_entropy(joint_positions))
            features_gen = joint_positions.reshape(joint_positions.shape[0], -1)
            fidelities.append(compute_fid(features_gen, reference_features))

    return {
        "penetrations": penetrations,
        "floatings": floatings,
        "slidings": slidings,
        "entropies": entropies,
        "fidelities": fidelities
    }

In [2]:
# Reference subject
reference_dir = "/home/mnt/datasets/183_retargeted/38.b3d"

In [3]:
# BIGE Metrics
bige_dir = "/home/mnt/generations/retargeted/BIGE"
bige_metrics = compute_metrics(bige_dir, reference_dir)

print(f"BIGE Penetration: {np.mean(bige_metrics['penetrations']):.4f} ± {np.std(bige_metrics['penetrations']):.4f}")
print(f"BIGE Floating: {np.mean(bige_metrics['floatings']):.4f} ± {np.std(bige_metrics['floatings']):.4f}")
print(f"BIGE Sliding: {np.mean(bige_metrics['slidings']):.4f} ± {np.std(bige_metrics['slidings']):.4f}")
print(f"BIGE Entropy: {np.mean(bige_metrics['entropies']):.4f} ± {np.std(bige_metrics['entropies']):.4f}")
print(f"BIGE Fidelity: {np.mean(bige_metrics['fidelities']):.4f} ± {np.std(bige_metrics['fidelities']):.4f}")

BIGE Penetration: nan ± nan
BIGE Floating: nan ± nan
BIGE Sliding: nan ± nan
BIGE Entropy: nan ± nan
BIGE Fidelity: nan ± nan


/opt/conda/envs/BIGE/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/envs/BIGE/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/envs/BIGE/lib/python3.8/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/conda/envs/BIGE/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/conda/envs/BIGE/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
# MDM Metrics
mdm_dir = "/home/mnt/generations/retargeted/MDM"
mdm_metrics = compute_metrics(mdm_dir, reference_dir, match_mode="looped")

print(f"MDM Penetration: {np.mean(mdm_metrics['penetrations']):.4f} ± {np.std(mdm_metrics['penetrations']):.4f}")
print(f"MDM Floating: {np.mean(mdm_metrics['floatings']):.4f} ± {np.std(mdm_metrics['floatings']):.4f}")
print(f"MDM Sliding: {np.mean(mdm_metrics['slidings']):.4f} ± {np.std(mdm_metrics['slidings']):.4f}")
print(f"MDM Entropy: {np.mean(mdm_metrics['entropies']):.4f} ± {np.std(mdm_metrics['entropies']):.4f}")
print(f"MDM Fidelity: {np.mean(mdm_metrics['fidelities']):.4f} ± {np.std(mdm_metrics['fidelities']):.4f}")

MDM Penetration: 0.0146 ± 0.0130
MDM Floating: 0.0184 ± 0.0264
MDM Sliding: 0.0320 ± 0.0207
MDM Entropy: 3.3385 ± 0.2133
MDM Fidelity: 0.4476 ± 0.4568


In [6]:
# PersonaBooth Metrics
persona_dir = "/home/mnt/generations/retargeted/PersonaBooth"
persona_metrics = compute_metrics(persona_dir, reference_dir, match_mode="grouped", group_size=3)

print(f"PersonaBooth Penetration: {np.mean(persona_metrics['penetrations']):.4f} ± {np.std(persona_metrics['penetrations']):.4f}")
print(f"PersonaBooth Floating: {np.mean(persona_metrics['floatings']):.4f} ± {np.std(persona_metrics['floatings']):.4f}")
print(f"PersonaBooth Sliding: {np.mean(persona_metrics['slidings']):.4f} ± {np.std(persona_metrics['slidings']):.4f}")
print(f"PersonaBooth Entropy: {np.mean(persona_metrics['entropies']):.4f} ± {np.std(persona_metrics['entropies']):.4f}")
print(f"PersonaBooth Fidelity: {np.mean(persona_metrics['fidelities']):.4f} ± {np.std(persona_metrics['fidelities']):.4f}")

PersonaBooth Penetration: 0.0139 ± 0.0088
PersonaBooth Floating: 0.0214 ± 0.0182
PersonaBooth Sliding: 0.0312 ± 0.0176
PersonaBooth Entropy: 3.4066 ± 0.1508
PersonaBooth Fidelity: 0.6916 ± 0.6820


In [6]:
# T2M-GPT Metrics
t2m_dir = "/home/mnt/generations/retargeted/T2M-GPT"
t2m_metrics = compute_metrics(t2m_dir, reference_dir, match_mode="grouped", group_size=3)

print(f"T2M-GPT Penetration: {np.mean(t2m_metrics['penetrations']):.4f} ± {np.std(t2m_metrics['penetrations']):.4f}")
print(f"T2M-GPT Floating: {np.mean(t2m_metrics['floatings']):.4f} ± {np.std(t2m_metrics['floatings']):.4f}")
print(f"T2M-GPT Sliding: {np.mean(t2m_metrics['slidings']):.4f} ± {np.std(t2m_metrics['slidings']):.4f}")
print(f"T2M-GPT Entropy: {np.mean(t2m_metrics['entropies']):.4f} ± {np.std(t2m_metrics['entropies']):.4f}")
print(f"T2M-GPT Fidelity: {np.mean(t2m_metrics['fidelities']):.4f} ± {np.std(t2m_metrics['fidelities']):.4f}")

Skipping trial 90: ref_idx 30 out of bounds
Skipping trial 91: ref_idx 30 out of bounds
Skipping trial 92: ref_idx 30 out of bounds
T2M-GPT Penetration: 0.0168 ± 0.0109
T2M-GPT Floating: 0.0213 ± 0.0175
T2M-GPT Sliding: 0.0511 ± 0.0309
T2M-GPT Entropy: 3.4736 ± 0.2182
T2M-GPT Fidelity: 0.5444 ± 0.5267


In [7]:
# Our metrics
results_dir = "/home/mnt/generations/results"
results_metrics = compute_metrics(results_dir, reference_dir, match_mode="grouped", group_size=3)
print(f"Results Penetration: {np.mean(results_metrics['penetrations']):.4f} ± {np.std(results_metrics['penetrations']):.4f}")
print(f"Results Floating: {np.mean(results_metrics['floatings']):.4f} ± {np.std(results_metrics['floatings']):.4f}")
print(f"Results Sliding: {np.mean(results_metrics['slidings']):.4f} ± {np.std(results_metrics['slidings']):.4f}")
print(f"Results Entropy: {np.mean(results_metrics['entropies']):.4f} ± {np.std(results_metrics['entropies']):.4f}")
print(f"Results Fidelity: {np.mean(results_metrics['fidelities']):.4f} ± {np.std(results_metrics['fidelities']):.4f}")

Results Penetration: 0.0697 ± 0.0346
Results Floating: 0.0670 ± 0.0324
Results Sliding: 0.0554 ± 0.0209
Results Entropy: 3.4448 ± 0.1023
Results Fidelity: 0.5809 ± 0.3593


In [ ]:
import nimblephysics as nimble
import numpy as np
import time

# Path to your generated B3D file
b3d_path = f"/home/mnt/generations/retargeted/MDM/mdm_motions.b3d"

# Load the B3D file
subject_on_disk = nimble.biomechanics.SubjectOnDisk(b3d_path)

# Load the Rajagopal skeleton
rajagopal_opensim = nimble.RajagopalHumanBodyModel()
skeleton = rajagopal_opensim.skeleton

# Visualize the first trial's motion
trial_idx = 0
num_frames = subject_on_disk.getTrialLength(trial_idx)
timestep = subject_on_disk.getTrialTimestep(trial_idx)

# Read all frames for the trial
frames = subject_on_disk.readFrames(trial=trial_idx, startFrame=0, numFramesToRead=num_frames)
poses = []
for frame in frames:
    # Use the first processing pass (kinematics)
    if len(frame.processingPasses) > 0:
        kinematics_pass = frame.processingPasses[0]
        if hasattr(kinematics_pass, "pos"):
            poses.append(np.array(kinematics_pass.pos))
poses = np.array(poses)  # shape: (num_frames, num_dofs)

# Create the GUI
gui = nimble.NimbleGUI()
gui.serve(8050)
gui.nativeAPI().renderSkeleton(skeleton)

# Animate the motion in a loop forever
try:
    while True:
        for t in range(len(poses)):
            skeleton.setPositions(poses[t])
            gui.nativeAPI().renderSkeleton(skeleton)
            time.sleep(timestep)
except KeyboardInterrupt:
    print("Keyboard interrupt received. Shutting down GUI session.")
    gui.shutdown()

GUIWebsocketServer will start serving a WebSocket server on ws://localhost:8070
Web GUI serving on http://localhost:9000


127.0.0.1 - - [31/Jul/2025 14:16:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Jul/2025 14:16:14] "GET /bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Jul/2025 14:16:15] "GET /favicon.ico HTTP/1.1" 200 -
[2025-07-31 14:16:37] [error] Handshake ended with HTTP error: 426
127.0.0.1 - - [31/Jul/2025 14:17:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Jul/2025 14:17:33] "GET /favicon.ico HTTP/1.1" 200 -


Keyboard interrupt received. Shutting down GUI session.


AttributeError: 'NimbleGUI' object has no attribute 'shutdown'